In [1]:
import tkinter as tk
import cv2
import mediapipe as mp
import time
from PIL import Image, ImageTk

class HandDetectorApp:
    def __init__(self, root, cap):
        self.root = root
        self.root.title("Hand Detector")
        self.cap = cap
        self.detector = handDetector()
        self.canvas = tk.Canvas(root, width=640, height=480)
        self.canvas.pack()
        self.fps_label = tk.Label(root, text="")
        self.fps_label.pack()
        self.highlight_button = tk.Button(root, text="Highlight Landmark", command=self.highlight_landmark)
        self.highlight_button.pack()
        self.landmark_entry = tk.Entry(root)
        self.landmark_entry.pack()
        self.landmark_idx = None
        self.update()

    def highlight_landmark(self):
        try:
            self.landmark_idx = int(self.landmark_entry.get())
        except ValueError:
            self.landmark_idx = None

    def update(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.flip(frame, 1)
            frame = cv2.resize(frame, (640, 480))

            frame = self.detector.findHands(frame)
            if self.landmark_idx is not None:
                lmList = self.detector.findPosition(frame)
                if len(lmList) > self.landmark_idx:
                    landmark = lmList[self.landmark_idx][1:]
                    self.detector.highlight_landmark(frame, landmark, (0, 255, 0))  # Highlight the selected landmark in green

            cTime = time.time()
            fps = 1 / (cTime - self.detector.pTime)
            self.detector.pTime = cTime

            cv2.putText(frame, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

            self.photo = ImageTk.PhotoImage(image=Image.fromarray(frame))
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)
            self.fps_label.config(text=f"FPS: {int(fps)}")

        self.root.after(1, self.update)


class handDetector:
    def __init__(self, mode=False, maxHands=2, model_complexity=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.model_complexity = model_complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.model_complexity, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.pTime = 0

    def findHands(self, img, draw=True): 
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB) 
        
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        lmList = []  
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                lmList.append([id, cx, cy])  
                if draw:
                    cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        return lmList

    def highlight_landmark(self, img, landmark, color):
        cx, cy = landmark
        cv2.circle(img, (cx, cy), 10, color, cv2.FILLED)

def main():
    cap = cv2.VideoCapture(1)
    root = tk.Tk()
    app = HandDetectorApp(root, cap)
    root.mainloop()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()